In [1]:
import tensorflow as tf
import keras
import pandas as pd
import cv2
import os

In [2]:
# Load data
training_data_df = pd.read_csv("./dataset/train/measurements.csv")
mapping_train_df = pd.read_csv("./dataset/train/subject_to_photo_map.csv")
testing_A_data_df = pd.read_csv("./dataset/testA/measurements.csv")
testing_B_data_df = pd.read_csv("./dataset/testB/measurements.csv")

In [3]:
# Data cleaning
mapping_train_clean = mapping_train_df.drop_duplicates(subset='subject_id', keep='first')
mapping_train_clean = mapping_train_clean.reset_index(drop=True)
mapping_train_clean

,subject_id,photo_id
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,e6f404ebda41ebe93573d3e219c88297
1,-5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc,69748a13010a169df0f7cb986ac1129e
2,-5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI,0d4a9cce6294a2bdc6dae93d0d9c42a8
3,-60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8,ecff10b191d87ec602eb58621d5a8e23
4,-67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY,ecb8c4b1031624047de1493c7e8b3905
...,...,...
2013,zqMxSbvyEBwZg0jFcfMR1B9fAq86pFKwQuOQ_Kjz_Lo,293cff7b27efa047a25e455593ed90d9
2014,zrydWW1YldsSPK0Dey0YMyjPCF0jOvwBTfofxzTnNQI,7a9bd49e515456d819cde0e3ce5aa4a0
2015,ztF-PzganuRaZfhRBc44RANbhV4ff1VVXHRe6MgDYrI,de46f1d8587111b668e1e5ed1b39ac5a
2016,ztYNUlq8M2d58t-fX1vsG5-JWNdZgtlUwYSBhDuryf8,34c2325b70ff2bb8ee68314b034f6642


In [4]:
training_data_df

,subject_id,ankle,arm-length,bicep,calf,chest,forearm,height,hip,leg-length,shoulder-breadth,shoulder-to-crotch,thigh,waist,wrist
0,-494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg,24.343954,52.902378,33.294395,38.709511,102.526024,27.599094,177.610992,108.047134,81.736435,35.915691,67.891273,57.872795,92.286171,16.709131
1,-5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc,22.001783,46.763233,28.288971,35.649963,99.253395,24.813375,164.379623,92.128685,75.127258,33.848778,60.541386,52.418320,87.307899,15.192038
2,-5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI,26.565006,53.288597,30.800959,39.854950,102.391273,27.719795,183.718842,106.044121,84.055527,37.856457,72.004379,52.934086,96.596413,17.936598
3,-60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8,24.779566,49.077751,34.613113,38.104267,111.164070,28.550409,169.059113,109.529465,74.310234,38.197800,68.536713,58.656673,103.395119,17.029860
4,-67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY,20.788853,42.229557,24.527142,29.181356,93.627777,21.979038,146.516632,87.134102,67.269859,31.659197,52.697277,41.844944,74.400856,15.120061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,zqMxSbvyEBwZg0jFcfMR1B9fAq86pFKwQuOQ_Kjz_Lo,28.004797,44.602837,38.303570,43.040047,122.899467,29.015333,158.808533,126.039978,72.811218,35.391495,61.160595,67.771896,115.621712,18.365078
2014,zrydWW1YldsSPK0Dey0YMyjPCF0jOvwBTfofxzTnNQI,22.504633,50.352051,25.394594,34.001297,87.909782,23.449825,164.984970,92.656151,76.770035,35.272739,58.375080,47.036442,78.589447,15.806385
2015,ztF-PzganuRaZfhRBc44RANbhV4ff1VVXHRe6MgDYrI,25.930092,53.487190,33.711838,36.684036,103.851021,28.664812,186.036514,104.019264,85.467796,38.296940,71.277969,54.600231,89.355942,18.517475
2016,ztYNUlq8M2d58t-fX1vsG5-JWNdZgtlUwYSBhDuryf8,25.591427,57.445339,30.275797,38.505508,101.348206,27.047409,191.739960,97.525414,90.530853,38.449238,71.265015,50.979385,86.367363,17.008245


In [5]:
merged_df = pd.merge(mapping_train_clean, training_data_df)
merged_df = merged_df.drop(['subject_id'], axis=1)
merged_df

,photo_id,ankle,arm-length,bicep,calf,chest,forearm,height,hip,leg-length,shoulder-breadth,shoulder-to-crotch,thigh,waist,wrist
0,e6f404ebda41ebe93573d3e219c88297,24.343954,52.902378,33.294395,38.709511,102.526024,27.599094,177.610992,108.047134,81.736435,35.915691,67.891273,57.872795,92.286171,16.709131
1,69748a13010a169df0f7cb986ac1129e,22.001783,46.763233,28.288971,35.649963,99.253395,24.813375,164.379623,92.128685,75.127258,33.848778,60.541386,52.418320,87.307899,15.192038
2,0d4a9cce6294a2bdc6dae93d0d9c42a8,26.565006,53.288597,30.800959,39.854950,102.391273,27.719795,183.718842,106.044121,84.055527,37.856457,72.004379,52.934086,96.596413,17.936598
3,ecff10b191d87ec602eb58621d5a8e23,24.779566,49.077751,34.613113,38.104267,111.164070,28.550409,169.059113,109.529465,74.310234,38.197800,68.536713,58.656673,103.395119,17.029860
4,ecb8c4b1031624047de1493c7e8b3905,20.788853,42.229557,24.527142,29.181356,93.627777,21.979038,146.516632,87.134102,67.269859,31.659197,52.697277,41.844944,74.400856,15.120061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,293cff7b27efa047a25e455593ed90d9,28.004797,44.602837,38.303570,43.040047,122.899467,29.015333,158.808533,126.039978,72.811218,35.391495,61.160595,67.771896,115.621712,18.365078
2014,7a9bd49e515456d819cde0e3ce5aa4a0,22.504633,50.352051,25.394594,34.001297,87.909782,23.449825,164.984970,92.656151,76.770035,35.272739,58.375080,47.036442,78.589447,15.806385
2015,de46f1d8587111b668e1e5ed1b39ac5a,25.930092,53.487190,33.711838,36.684036,103.851021,28.664812,186.036514,104.019264,85.467796,38.296940,71.277969,54.600231,89.355942,18.517475
2016,34c2325b70ff2bb8ee68314b034f6642,25.591427,57.445339,30.275797,38.505508,101.348206,27.047409,191.739960,97.525414,90.530853,38.449238,71.265015,50.979385,86.367363,17.008245


In [6]:
# Target / labels
target_columns = ['bicep', 'thigh', 'waist']
y = merged_df[target_columns]
y_array = y.to_numpy()
y.loc[150]

bicep    25.842754
thigh    48.917801
waist    80.946945
Name: 150, dtype: float64

In [7]:
# Load images data
images = []

for index, row in mapping_train_clean.iterrows():
    photo_id = row["photo_id"]
    img_path = os.path.join("./dataset/train/mask/", f'{photo_id}.png')
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    images.append(img)

In [8]:
import numpy as np
# Convert images array to NumPy array  
images = np.array(images)
images = images / 255.


In [9]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
images[100]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(960, 720, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu')
])
model.summary()
model.compile(loss="mean_squared_error", optimizer='adam', metrics=['mae'])
model.fit(images, y_array, epochs=5, batch_size=20)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 958, 718, 32)      320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 479, 359, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 477, 357, 32)      9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 238, 178, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 236, 176, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 118, 88, 64)     

In [ ]:
tes_prediksi = np.expand_dims(images[105], axis=0)
print(model.predict(tes_prediksi))
print(y_array[105])

1/1 [==============================] - 0s 94ms/step
[[22.796745 48.856052 26.37998 ]]
[22.55992317 50.42573547 24.32812119]
